# CS 340 - Module 5: Full CRUD and Indexing Tests (Grazioso Salvare)

This notebook validates the full CRUD functionality and indexing for the AnimalShelter module used in the Grazioso Salvare project.

What you'll do:
1) Configure environment and connect with the `aacuser` credentials
2) Ensure recommended indexes exist
3) Execute end-to-end CRUD tests (Create, Read, Update, Delete)
4) Capture screenshots for your Project One README submission

Important: Before running, set your environment variables or edit the configuration cell below to use `aacuser` and your MongoDB host/port.

Screenshots to capture for your README:
- MongoDB import command and execution output (from earlier milestone)
- MongoDB authentication as `aacuser` (mongosh screen from earlier milestone)
- Successful outputs from the CRUD test cells in this notebook (Create/Read/Update/Delete)

Note on database name casing: This project uses database name `aac` (lowercase) by default to match the project codebase configuration.

In [ ]:
# BOOTSTRAP & IMPORT: Find project root and import AnimalShelter
import sys
from pathlib import Path

candidate = Path.cwd()
project_root = None
for _ in range(6):
    if (candidate / "animal_shelter").is_dir():
        project_root = candidate
        break
    if (candidate / "m4-m5" / "animal_shelter").is_dir():
        project_root = candidate / "m4-m5"
        break
    candidate = candidate.parent

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("Project root added to sys.path:", project_root)

if not project_root:
    raise RuntimeError("Could not find the 'animal_shelter' directory. Please run this notebook from within the project structure.")

from animal_shelter import AnimalShelter
print("Successfully imported AnimalShelter module.")

In [ ]:
# Configuration: You may modify these if needed or set them as environment variables beforehand.
import os

# Prefer environment variables. If not set, fallback to these values.
os.environ.setdefault("AAC_DATABASE", "aac")
os.environ.setdefault("AAC_COLLECTION",   "animals")

# REQUIRED for grading: use the aacuser account created in Module 3
os.environ.setdefault("AAC_USER", "aacuser")
os.environ.setdefault("AAC_PASS", "SECRET")  # Replace with your actual aacuser password

# Apporto example (adjust if needed). You can also set MONGO_HOST/MONGO_PORT explicitly.
os.environ.setdefault("MONGO_HOST", os.getenv("MONGODB_HOST", "localhost"))
os.environ.setdefault("MONGO_PORT", os.getenv("MONGODB_PORT", "27017"))

print("Configuration loaded:")
print(" AAC_DATABASE=", os.getenv("AAC_DATABASE"))
print(" AAC_COLLECTION=", os.getenv("AAC_COLLECTION"))
print(" MONGO_HOST=", os.getenv("MONGO_HOST"))
print(" MONGO_PORT=", os.getenv("MONGO_PORT"))
print(" Using aacuser? =>", os.getenv("AAC_USER") == "aacuser")

## Instantiate the AnimalShelter class

This uses your environment variables. On successful connection, you'll see a log message indicating the MongoDB host and port, and the collection setup.

In [ ]:
# Instantiate and verify connection
shelter = AnimalShelter()
stats = shelter.get_collection_stats()
print("Connected. Collection stats:")
print(stats)

## Ensure Indexes

Creates recommended indexes to improve query performance. This is safe to run multiple times. Recommended indexes:
- animal_id
- animal_type
- breed
- outcome_type
- age_upon_outcome

If you haven't already, capture a screenshot of this step running successfully to demonstrate indexing for Project One.

In [ ]:
shelter.ensure_indexes()
print("Indexes ensured.")

## Create: Insert a test document

We'll insert a unique test record and assert success. Capture a screenshot of this cell's output.

Note: We use a predictable `animal_id` with a timestamp suffix to avoid collisions across runs.

In [ ]:
import time

unique_suffix = str(int(time.time()))
TEST_ANIMAL_ID = f"M5-TEST-{unique_suffix}"

test_doc = {
    "animal_id": TEST_ANIMAL_ID,
    "name": "Ranger",
    "animal_type": "Dog",
    "breed": "Labrador Retriever",
    "age_upon_outcome": "1 year",
    "outcome_type": ""
}

created = shelter.create(test_doc)
print("Create success?", created)
assert created is True, "Create should return True for a successful insert"

## Read: Retrieve the document we just created

We query by `animal_id` and confirm that we get back exactly one document with the expected fields. Capture a screenshot of this output as part of your CRUD demonstration.

In [ ]:
docs = shelter.read({"animal_id": TEST_ANIMAL_ID})
print(f"Read returned {len(docs)} document(s)")
assert isinstance(docs, list), "Read should return a list"
assert len(docs) == 1, "Expected exactly one matching document"
print("Document:", {k: v for k, v in docs[0].items() if k != "_id"})  # hide ObjectId for readability

## Update: Modify the test document

Project One requires the Update method to return the number of modified documents. We'll change the `outcome_type` and confirm the modified count. Then, we'll verify by reading again.

Capture a screenshot of the modified count and the verification read.

In [ ]:
modified_count = shelter.update(
    {"animal_id": TEST_ANIMAL_ID},
    {"$set": {"outcome_type": "Adoption"}},
    many=False
)
print("Modified count:", modified_count)
assert modified_count in (0, 1), "Modified count should be 0 or 1 depending on server version and prior state"

post_update_docs = shelter.read({"animal_id": TEST_ANIMAL_ID})
print("Post-update document:", {k: v for k, v in post_update_docs[0].items() if k != "_id"})
assert post_update_docs[0]["outcome_type"] == "Adoption", "Update did not persist as expected"

## Delete: Remove the test document

Project One requires the Delete method to return the number of deleted objects. We'll delete the test document and then verify it no longer exists.

Capture a screenshot of the deleted count and the verification read with zero results.

In [ ]:
deleted_count = shelter.delete({"animal_id": TEST_ANIMAL_ID}, many=False)
print("Deleted count:", deleted_count)
assert deleted_count in (0, 1), "Deleted count should be 0 or 1 depending on prior state"

verify_delete = shelter.read({"animal_id": TEST_ANIMAL_ID})
print(f"Read after delete returned {len(verify_delete)} document(s)")
assert len(verify_delete) == 0, "Document should have been deleted"

## Bulk (Many) Operations (Optional)

If you want to demonstrate update_many/delete_many for extra credit or internal validation, you can duplicate this pattern with many=True. This is optional for the core rubric but can be illustrative for performance/behavior understanding.

Example idea:
- Read a small subset of dogs by breed or outcome_type and update a temporary field for those, then delete it.

Be careful not to modify large portions of the dataset in shared environments.

In [ ]:
# OPTIONAL: Example pattern (commented to avoid accidental mass updates)
#
# criteria = {"animal_type": "Dog", "breed": {"$regex": "Labrador", "$options": "i"}}
# update_many_count = shelter.update(criteria, {"$set": {"temp_m5_flag": True}}, many=True)
# print("update_many modified:", update_many_count)
#
# revert_many_count = shelter.update(criteria, {"$unset": {"temp_m5_flag": ""}}, many=True)
# print("revert_many modified:", revert_many_count)
pass

## Collection Statistics and Wrap-Up

Useful for documentation and verifying the collection state after tests. Capture a screenshot of this summary if helpful for your README.

In [ ]:
final_stats = shelter.get_collection_stats()
print("Final Collection Stats:")
print(final_stats)

# Always a good practice to close the connection when you're done
shelter.close_connection()
print("Connection closed.")

## Screenshot Prompts (for README / Word submission)

Include the following screenshots in your Project One submission:
1) `mongoimport` command and execution output (from Module 3 or fresh import)
2) `mongosh` authentication as `aacuser` and switching to the `aac` database
3) From this notebook:
   - Configuration cell output (showing use of `aacuser`)
   - Successful connection and initial collection stats
   - Index creation confirmation
   - Create: success output
   - Read: one-document result with your test `animal_id`
   - Update: modified count and verification read showing updated fields
   - Delete: deleted count and verification read showing zero documents
   - Final collection stats

This set meets the rubric’s demonstration requirements for full CRUD functionality and indexing.